In [161]:
import numpy as np
import scanpy as sc
import pandas as pd

In [162]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.9.1 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.8


Read in the AnnData file.

In [163]:
adata = sc.read_h5ad("/Volumes/NAME/deconvolution-project/filtered_MS_nuclei_hashing_gx12.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 4889 × 33694
    var: 'gene_ids', 'feature_types'


In [164]:
adata.var

,gene_ids,feature_types
RP11-34P13.3,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression
OR4F5,ENSG00000186092,Gene Expression
RP11-34P13.7,ENSG00000238009,Gene Expression
RP11-34P13.8,ENSG00000239945,Gene Expression
...,...,...
AC233755.2,ENSG00000277856,Gene Expression
AC233755.1,ENSG00000275063,Gene Expression
AC240274.1,ENSG00000271254,Gene Expression
AC213203.1,ENSG00000277475,Gene Expression


In [165]:
gene_expression_MBP = adata[:, "MBP"].X.toarray()

In [166]:
gene_expression_MBP

array([[ 3.],
       [59.],
       [ 2.],
       ...,
       [51.],
       [ 6.],
       [23.]], dtype=float32)

In [167]:
# Convert the numpy array to a pandas DataFrame
new_df = pd.DataFrame(gene_expression_MBP, index=adata.obs_names, columns=["gene_expression_MBP"])

# Concatenate the new DataFrame with the existing adata.obs DataFrame
adata.obs = pd.concat([adata.obs, new_df], axis=1)


In [168]:
import scipy.io

# Load the sparse matrix from the Matrix Market file
decontamination_matrix = scipy.io.mmread("/Volumes/NAME/deconvolution-project/gx12_decontaminated_matrix.mtx")

In [169]:
decontamination_matrix = decontamination_matrix.tocsr()
before_decontamination_matrix = adata.X

In [170]:
import numpy as np
from scipy.sparse import csr_matrix

# assume matrix_a and matrix_b are two sparse matrices in CSR format with the same shape
# first convert them to arrays with the same shape and fill missing values with zeros
a = before_decontamination_matrix.toarray()
b = decontamination_matrix.T.toarray()
a[np.isnan(a)] = 0
b[np.isnan(b)] = 0

# subtract matrix_a from matrix_b element-wise
result = a - b

# convert the result back to CSR format
sub_decontamination_matrix = csr_matrix(result)


In [171]:
# somehow represent these as percentage points
# maybe just do a separate obs column? Need to find the gene values though --> should be in the matrix of the original adata

In [172]:
adata.layers["counts"] = adata.X
adata.layers["soupX_counts"] = sub_decontamination_matrix
adata.X = adata.layers["soupX_counts"]

In [173]:
adata.X

<4889x33694 sparse matrix of type '<class 'numpy.float64'>'
	with 13475030 stored elements in Compressed Sparse Row format>

In [174]:
adata.layers["soupX_counts"] = adata.layers["soupX_counts"].tocsr()

In [175]:
adata.X

<4889x33694 sparse matrix of type '<class 'numpy.float64'>'
	with 13475030 stored elements in Compressed Sparse Row format>

In [176]:
gene_expression_MBP_sub = adata[:, "MBP"].X.toarray()

In [177]:
gene_expression_MBP_sub

array([[3.        ],
       [1.03717443],
       [0.26282582],
       ...,
       [5.5402204 ],
       [6.        ],
       [0.29361805]])

In [178]:
# Convert the numpy array to a pandas DataFrame
new_df = pd.DataFrame(gene_expression_MBP_sub, index=adata.obs_names, columns=["gene_expression_MBP_sub"])

# Concatenate the new DataFrame with the existing adata.obs DataFrame
adata.obs = pd.concat([adata.obs, new_df], axis=1)


In [179]:
adata.obs

,gene_expression_MBP,gene_expression_MBP_sub
AAACCCAAGGTGTGAC-1,3.0,3.000000
AAACCCAGTGAGAGGG-1,59.0,1.037174
AAACGAAAGAATCTAG-1,2.0,0.262826
AAACGAACACATATGC-1,3.0,0.232459
AAACGAACACGACTAT-1,46.0,1.358648
...,...,...
TTTGTTGAGTCCCAGC-1,4.0,0.088132
TTTGTTGCAGCGCGTT-1,71.0,0.594395
TTTGTTGGTTATCTTC-1,51.0,5.540220
TTTGTTGTCCAGTGTA-1,6.0,6.000000


In [180]:
adata.obs["gene_expression_MBP_ratio_changed"] = adata.obs["gene_expression_MBP_sub"] / adata.obs["gene_expression_MBP"]

In [181]:
adata.obs

,gene_expression_MBP,gene_expression_MBP_sub,gene_expression_MBP_ratio_changed
AAACCCAAGGTGTGAC-1,3.0,3.000000,1.000000
AAACCCAGTGAGAGGG-1,59.0,1.037174,0.017579
AAACGAAAGAATCTAG-1,2.0,0.262826,0.131413
AAACGAACACATATGC-1,3.0,0.232459,0.077486
AAACGAACACGACTAT-1,46.0,1.358648,0.029536
...,...,...,...
TTTGTTGAGTCCCAGC-1,4.0,0.088132,0.022033
TTTGTTGCAGCGCGTT-1,71.0,0.594395,0.008372
TTTGTTGGTTATCTTC-1,51.0,5.540220,0.108632
TTTGTTGTCCAGTGTA-1,6.0,6.000000,1.000000


In [182]:
adata.obs.to_csv("/Volumes/NAME/deconvolution-project/adata_obs.csv", index=True)

Read in the metadata and the doublet and donor labels by Xichen.

In [129]:
obs = pd.read_csv(
    "/Volumes/NAME/deconvolution-project/gx12_final_result_all.csv")
obs

,Barcode,donor_identity
0,ACTTATCTCATGCCAA-1,Hash453-TotalSeqA
1,GGGTCACAGCAGCAGT-1,Hash452-TotalSeqA
2,CAGAGCCGTTGGGATG-1,Hash451-TotalSeqA
3,GGTCACGTCGGTCTAA-1,Hash452-TotalSeqA
4,GACATCACATCAGCGC-1,Hash455-TotalSeqA
...,...,...
4884,ATGATCGTCATGCCAA-1,Hash453-TotalSeqA
4885,CCTCATGTCAAACGTC-1,Hash456-TotalSeqA
4886,TGTTACTCAAGACTGG-1,Hash455-TotalSeqA
4887,AACCACATCATGTCTT-1,Hash453-TotalSeqA


In [130]:
obs_vario = pd.read_csv(
    "/Volumes/NAME/deconvolution-project/gx12_vireo_assignment.csv", 
    usecols=["cell", "donor_id", "classification"])
obs_vario

,cell,donor_id,classification
0,AAACCCAAGGTGTGAC-1,donor1,singlet
1,AAACCCAGTGAGAGGG-1,donor2,singlet
2,AAACGAAAGAATCTAG-1,donor5,singlet
3,AAACGAACACATATGC-1,donor5,singlet
4,AAACGAACACGACTAT-1,donor5,singlet
...,...,...,...
4884,TTTGTTGAGTCCCAGC-1,donor2,singlet
4885,TTTGTTGCAGCGCGTT-1,doublet,doublet
4886,TTTGTTGGTTATCTTC-1,doublet,doublet
4887,TTTGTTGTCCAGTGTA-1,donor0,singlet


In [131]:
obs_meta_sel = pd.read_csv(
    "/Volumes/NAME/deconvolution-project/adataobs_MSmultiplex_DendrouSchirmerJakel_integrated.csv", 
    usecols=["cluster_label", "lowres_cluster_label", "sample_id", "batch"])
obs_meta_sel.head()

,sample_id,batch,cluster_label,lowres_cluster_label
0,704193_GX53,0,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro
1,704193_GX53,0,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro
2,704193_GX53,0,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro
3,704193_GX53,0,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro
4,704193_GX53,0,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro


In [132]:
obs_meta_all = pd.read_csv("/Volumes/NAME/deconvolution-project/adataobs_MSmultiplex_DendrouSchirmerJakel_integrated.csv", index_col=0, low_memory=False)
obs_meta_all.head()

,sample_id,batch,antibody,library_id,patient,block_position,demult_sample_id,disorder,disease_type,age,...,tissue,library,brain_region,leiden_res_0.1,leiden_res_0.3,leiden_res_0.5,leiden_res_1,cluster_label,lowres_cluster_label,bucket
AAACGCTTCGTAGGAG-1-704193_GX53-dendrou,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,43.0,...,NaN,NaN,NaN,2,2,2,1,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro,astrocytes
AAAGGATTCATGCGGC-1-704193_GX53-dendrou,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,43.0,...,NaN,NaN,NaN,2,2,2,1,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro,astrocytes
AAAGGATTCGCTTAAG-1-704193_GX53-dendrou,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,43.0,...,NaN,NaN,NaN,2,2,2,1,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro,astrocytes
AAAGGTAAGGCCTTCG-1-704193_GX53-dendrou,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,43.0,...,NaN,NaN,NaN,2,2,2,1,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro,astrocytes
AACAAAGTCCGATAAC-1-704193_GX53-dendrou,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,43.0,...,NaN,NaN,NaN,2,2,2,1,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro,astrocytes


Set Index of metadata as a column. Cut all information after barcode.

In [133]:
obs_meta_all.reset_index(inplace=True)
obs_meta_all['index'] = obs_meta_all['index'].str[:18]
obs_meta_all.head()

,index,sample_id,batch,antibody,library_id,patient,block_position,demult_sample_id,disorder,disease_type,...,tissue,library,brain_region,leiden_res_0.1,leiden_res_0.3,leiden_res_0.5,leiden_res_1,cluster_label,lowres_cluster_label,bucket
0,AAACGCTTCGTAGGAG-1,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,...,NaN,NaN,NaN,2,2,2,1,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro,astrocytes
1,AAAGGATTCATGCGGC-1,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,...,NaN,NaN,NaN,2,2,2,1,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro,astrocytes
2,AAAGGATTCGCTTAAG-1,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,...,NaN,NaN,NaN,2,2,2,1,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro,astrocytes
3,AAAGGTAAGGCCTTCG-1,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,...,NaN,NaN,NaN,2,2,2,1,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro,astrocytes
4,AACAAAGTCCGATAAC-1,704193_GX53,0,NaN,704193_GX53,MS352,NaN,704193_GX53_0,ms,SP,...,NaN,NaN,NaN,2,2,2,1,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro,astrocytes


Filter by gx12 in sample_id (not case-sensitive).

In [134]:
obs_meta_all_gx12 = obs_meta_all[obs_meta_all['sample_id'].str.contains('gx12', case=False)]
obs_meta_all_gx12

,index,sample_id,batch,antibody,library_id,patient,block_position,demult_sample_id,disorder,disease_type,...,tissue,library,brain_region,leiden_res_0.1,leiden_res_0.3,leiden_res_0.5,leiden_res_1,cluster_label,lowres_cluster_label,bucket
411,AAAGGGCCAATTTCCT-1,747495_GX12,1,Hash453,747495_GX12,C005,P2D4,747495_GX12_3,control,none,...,NaN,NaN,NaN,2,2,2,1,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro,astrocytes
412,AACAAGACAACTGGTT-1,747495_GX12,1,Hash451,747495_GX12,C014_d,P5C3,747495_GX12_1,control,none,...,NaN,NaN,NaN,2,2,2,1,PhagoProtoAstro_MERTKhi_MEGF10hi,ProtoAstro,astrocytes
413,AACCACAGTTCCGGTG-1,747495_GX12,1,Hash451,747495_GX12,C014_d,P5C3,747495_GX12_1,control,none,...,NaN,NaN,NaN,2,2,2,1,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro,astrocytes
414,AACCCAACACTCTCGT-1,747495_GX12,1,Hash456,747495_GX12,MS425,A3A2,747495_GX12_6,ms,SP,...,NaN,NaN,NaN,2,2,2,1,ProtoAstro_SLC1A2hi_NRXN1hi,ProtoAstro,astrocytes
415,AAGCATCAGAGTCTTC-1,747495_GX12,1,Hash454,747495_GX12,MS438,P2D4,747495_GX12_4,ms,Undetermined,...,NaN,NaN,NaN,2,2,2,1,FibrousAstro-CD44hi_S100Bpos_FOShi_VIMpos,FibrousAstro,astrocytes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44550,TTTCGATCAGTTAGGG-1,747495_GX12,1,Hash454,747495_GX12,MS438,P2D4,747495_GX12_4,ms,Undetermined,...,NaN,NaN,NaN,1,1,1,2,CUX2_LAMP5_YWHAHhi,CUX2,neurons
44551,TTTCGATGTGTCGATT-1,747495_GX12,1,Hash451,747495_GX12,C014_d,P5C3,747495_GX12_1,control,none,...,NaN,NaN,NaN,9,10,9,8,FEZF2_SEMA3Ehi,FEZF2,neurons
44552,TTTGGAGTCGAGAATA-1,747495_GX12,1,Hash451,747495_GX12,C014_d,P5C3,747495_GX12_1,control,none,...,NaN,NaN,NaN,3,9,8,9,RORB_TOXhi_ARHGAP15hi,RORB,neurons
44553,TTTGGTTAGATAGGGA-1,747495_GX12,1,Hash455,747495_GX12,C064,A2D1,747495_GX12_5,control,none,...,NaN,NaN,NaN,5,7,6,6,VIP_HGFAC,VIP,neurons


Merge Xichen's vireo data with the new data to get the correct order.

In [135]:
obs_ordered = pd.merge(obs_vario, obs, left_on='cell', right_on='Barcode', how='outer')
obs_ordered

,cell,donor_id,classification,Barcode,donor_identity
0,AAACCCAAGGTGTGAC-1,donor1,singlet,AAACCCAAGGTGTGAC-1,Hash451-TotalSeqA
1,AAACCCAGTGAGAGGG-1,donor2,singlet,AAACCCAGTGAGAGGG-1,Hash453-TotalSeqA
2,AAACGAAAGAATCTAG-1,donor5,singlet,AAACGAAAGAATCTAG-1,Hash452-TotalSeqA
3,AAACGAACACATATGC-1,donor5,singlet,AAACGAACACATATGC-1,Hash452-TotalSeqA
4,AAACGAACACGACTAT-1,donor5,singlet,AAACGAACACGACTAT-1,Hash452-TotalSeqA
...,...,...,...,...,...
4884,TTTGTTGAGTCCCAGC-1,donor2,singlet,TTTGTTGAGTCCCAGC-1,Hash453-TotalSeqA
4885,TTTGTTGCAGCGCGTT-1,doublet,doublet,TTTGTTGCAGCGCGTT-1,doublet
4886,TTTGTTGGTTATCTTC-1,doublet,doublet,TTTGTTGGTTATCTTC-1,doublet
4887,TTTGTTGTCCAGTGTA-1,donor0,singlet,TTTGTTGTCCAGTGTA-1,Hash454-TotalSeqA


Merge Xichens obs information and the metadata for gx12. Not all cells have a match in the gx12. Here NaN is put in for those columns.

In [136]:
merged_obs = pd.merge(obs_ordered, obs_meta_all_gx12, left_on='cell', right_on='index', how='outer')
merged_obs

,cell,donor_id,classification,Barcode,donor_identity,index,sample_id,batch,antibody,library_id,...,tissue,library,brain_region,leiden_res_0.1,leiden_res_0.3,leiden_res_0.5,leiden_res_1,cluster_label,lowres_cluster_label,bucket
0,AAACCCAAGGTGTGAC-1,donor1,singlet,AAACCCAAGGTGTGAC-1,Hash451-TotalSeqA,AAACCCAAGGTGTGAC-1,747495_GX12,1,Hash451,747495_GX12,...,NaN,NaN,NaN,1.0,1.0,1.0,2.0,CUX2_PALMDpos_FREM3pos,CUX2,neurons
1,AAACCCAGTGAGAGGG-1,donor2,singlet,AAACCCAGTGAGAGGG-1,Hash453-TotalSeqA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAACGAAAGAATCTAG-1,donor5,singlet,AAACGAAAGAATCTAG-1,Hash452-TotalSeqA,AAACGAAAGAATCTAG-1,747495_GX12,1,Hash452,747495_GX12,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,OG - OPALINlow_SLC5A11hi_RBFOXhi,OG - OPALINlow_SLC5A11hi_RBFOXhi,oligodendrocytes
3,AAACGAACACATATGC-1,donor5,singlet,AAACGAACACATATGC-1,Hash452-TotalSeqA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACGAACACGACTAT-1,donor5,singlet,AAACGAACACGACTAT-1,Hash452-TotalSeqA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4884,TTTGTTGAGTCCCAGC-1,donor2,singlet,TTTGTTGAGTCCCAGC-1,Hash453-TotalSeqA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4885,TTTGTTGCAGCGCGTT-1,doublet,doublet,TTTGTTGCAGCGCGTT-1,doublet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4886,TTTGTTGGTTATCTTC-1,doublet,doublet,TTTGTTGGTTATCTTC-1,doublet,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4887,TTTGTTGTCCAGTGTA-1,donor0,singlet,TTTGTTGTCCAGTGTA-1,Hash454-TotalSeqA,TTTGTTGTCCAGTGTA-1,747495_GX12,1,Hash454,747495_GX12,...,NaN,NaN,NaN,1.0,1.0,1.0,2.0,CUX2_PALMDpos_FREM3pos,CUX2,neurons


Changing names of some columns to paper_X to not interfere with later column generation.

In [137]:
merged_obs = merged_obs.rename(columns={'doublet_scores' : 'paper_doublet_scores', 'predicted_doublets': 'paper_predicted_doublets'})

Adding only a few selected columns to not interfere with later doubled column labeling by further analysis.

In [138]:
merged_obs_sel = merged_obs.loc[:, ["Barcode", "donor_identity", "donor_id", "cluster_label", "lowres_cluster_label", "sample_id", "bucket", "disorder", "paper_doublet_scores", "paper_predicted_doublets"]]
merged_obs_sel.set_index('Barcode', inplace=True)
merged_obs_sel

,donor_identity,donor_id,cluster_label,lowres_cluster_label,sample_id,bucket,disorder,paper_doublet_scores,paper_predicted_doublets
Barcode,,,,,,,,,
AAACCCAAGGTGTGAC-1,Hash451-TotalSeqA,donor1,CUX2_PALMDpos_FREM3pos,CUX2,747495_GX12,neurons,control,0.043699,False
AAACCCAGTGAGAGGG-1,Hash453-TotalSeqA,donor2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACGAAAGAATCTAG-1,Hash452-TotalSeqA,donor5,OG - OPALINlow_SLC5A11hi_RBFOXhi,OG - OPALINlow_SLC5A11hi_RBFOXhi,747495_GX12,oligodendrocytes,ms,0.031493,False
AAACGAACACATATGC-1,Hash452-TotalSeqA,donor5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACGAACACGACTAT-1,Hash452-TotalSeqA,donor5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
TTTGTTGAGTCCCAGC-1,Hash453-TotalSeqA,donor2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TTTGTTGCAGCGCGTT-1,doublet,doublet,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TTTGTTGGTTATCTTC-1,doublet,doublet,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
adata.obs = merged_obs_sel
adata.obs.head()

ValueError: Length of passed value for obs_names is 4889, but this AnnData has shape: (108, 21006)

In [ ]:
adata

In [ ]:
adata.var

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata.shape

Doublet score generation via scrublet.

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.X)
doublet_scores, predicted_doublets = scrub.scrub_doublets()

In [ ]:
print(doublet_scores)

In [ ]:
adata.obs['doublet_score'] = doublet_scores

Start of filtering and QC.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
print(type(adata.X))
print(type(adata.layers["counts"]))
print(type(adata.layers["soupX_counts"]))
adata.layers["soupX_counts"] = adata.layers["soupX_counts"].tocsr()

In [ ]:
print(type(adata.X))
print(type(adata.layers["counts"]))
print(type(adata.layers["soupX_counts"]))

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

Check for the probable cutoff used in the dataset by the previous MS analysis. (10%, supported by their methods section)

In [ ]:
print(obs_meta_all_gx12["pct_counts_mt"].max())
print(obs_meta_all_gx12["n_genes_by_counts"].max())

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 10849, :] # adjusted 2500 to 10894 by previous analysis
adata = adata[adata.obs.pct_counts_mt < 10, :] # under 10 percent as probable cutoff

In [ ]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color='CST3')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# adata.write(results_file)

In [ ]:
adata

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
sc.tl.umap(adata)

In [ ]:
print(adata.obs)

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.tl.leiden(adata, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(adata, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(adata, key_added="leiden_res1", resolution=1.0)
sc.tl.leiden(adata, key_added="leiden_res2", resolution=2.0)

In [ ]:
sc.pl.umap(adata, color=['doublet_score', 'n_genes_by_counts', 'n_genes', 'donor_identity'])

Add a column to adata.obs that only shows doublet vs. no doublet.

In [ ]:
doublet_boolean = adata.obs['donor_identity'].str.contains("doublet")
adata.obs['doublet_classification_pipeline'] = ['doublet' if result else 'no doublet' for result in doublet_boolean]

In [ ]:
adata.obs

In [ ]:
sc.pl.umap(adata, color=['doublet_score', 'n_genes_by_counts', 'n_genes', 'doublet_classification_pipeline'])

Generating a column that labels all NaN cells in the bucket column as new. Then combine these with the leiden_clustering.

In [ ]:
adata.obs['newold'] = np.where(pd.isna(adata.obs['bucket']), 'new', 'old')
adata.obs["leiden_new_old"]=adata.obs['leiden'].str.cat(adata.obs['newold'], sep='_')
adata.obs["leiden_res0_25_new_old"]=adata.obs['leiden_res0_25'].str.cat(adata.obs['newold'], sep='_')

Plotting Violin plots to compare the new and paper given doublet scores.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

paper_doublet_scores = adata.obs["paper_doublet_scores"]

doublet_scores_df = pd.DataFrame({'paper_doublet_scores': paper_doublet_scores, 'total_new_doublet_scores': adata.obs["doublet_score"]})

plt.figure(figsize=(8, 6))
sns.violinplot(data=doublet_scores_df, save="violet_doublets_all_cells.png")
plt.ylabel('Doublet Score')
plt.title('Comparison of Doublet Scores between paper scores and newly generated scores with new cells included')

In [ ]:
adata_old = adata[adata.obs['newold'] == 'old']

data_to_plot = adata_old.obs.loc[:, ['paper_doublet_scores', 'doublet_score']]

plt.figure(figsize=(8, 6))
sns.violinplot(data=data_to_plot, save="violet_doublets_old_cells.png")
plt.ylabel('Doublet Score')
plt.title('Comparison of Doublet Scores between paper scores and newly generated scores without new cells included')

In [ ]:
paper_doublet_scores = adata.obs["paper_doublet_scores"]

doublet_scores_df_2 = pd.DataFrame({'paper_doublet_scores': paper_doublet_scores, 'total_new_doublet_scores': adata.obs["doublet_score"]})

plt.figure(figsize=(8, 6))
sns.violinplot(data=doublet_scores_df, save=".png")
plt.ylabel('Doublet Score')
plt.title('Comparison of Doublet Scores between paper scores and newly generated scores with new cells included')

Plotting the cell type description the paper gave over all cells including the new ones to identify clusters.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata, color=["bucket"], ax=ax))

Plotting a donor_id colored umap.

In [ ]:
sc.pl.umap(adata, save=".png", color=["donor_identity", "donor_id"])

Excluding cells that had no donor_id and plot them.

In [ ]:
adata_donor_assigned = adata[~(adata.obs['donor_identity'] == 'negative'), :]
print(adata_donor_assigned.obs)
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata_donor_assigned, color=["bucket"], ax=ax))

Used Xichens doublet prediction to filter out doublets.

In [ ]:
adata_donor_assigned_no_doublets = adata_donor_assigned[~(adata_donor_assigned.obs['donor_identity'] == 'doublet'), :]
print(adata_donor_assigned_no_doublets)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata_donor_assigned_no_doublets, color=["bucket"], ax=ax))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata_donor_assigned_no_doublets, color=["lowres_cluster_label"], ax=ax))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata_donor_assigned_no_doublets, color=["cluster_label"], ax=ax))

Plotting doublet parameters after removal of Xichens doublets and unassigned.

In [ ]:
sc.pl.umap(adata_donor_assigned_no_doublets, color=['doublet_score', 'n_genes_by_counts', 'n_genes', 'doublet_classification_pipeline'])

Plotting leiden clusters.

In [ ]:
sc.pl.umap(adata, color=['leiden'])

Showing ranked genes by t-test.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.settings.verbosity = 2  # reduce the verbosity

Showing ranked genes by wilcoxon.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

Plotting different leiden resolutions.

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res0_25", "leiden_res0_5", "leiden_res1", "leiden_res2"],
    legend_loc="on data",
)

Looking more closely at the new leiden cluster 18 with QC parameters. 

In [ ]:
cluster_18 = adata.obs[adata.obs['leiden'] == '18']

In [ ]:
cluster_18['doublet_score'].describe()

In [ ]:
adata.obs['doublet_score'].describe()

In [ ]:
cluster_18['n_genes_by_counts'].describe()

In [ ]:
adata.obs['n_genes_by_counts'].describe()

In [ ]:
cluster_18['n_genes'].describe()

In [ ]:
adata.obs['n_genes'].describe()

Showing that there are no NaN values or NA in the original metadata from the gx12-cells in the 'bucket' category. This means that all NaN values (all grey cells in the umap) are new cells.

In [ ]:
number_nan_meta_all = obs_meta_all['bucket'].isna().sum()
print('Number of NaN values:', number_nan_meta_all)

In [ ]:
unique_values_gx12_bucket = obs_meta_all_gx12['bucket'].unique()
print(unique_values_gx12_bucket)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_res0_25', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
neurons_old = adata_donor_assigned_no_doublets[(adata_donor_assigned_no_doublets.obs['bucket'] == 'neurons'), :]
print(neurons_old)

neurons_new_1 = adata_donor_assigned_no_doublets[(adata_donor_assigned_no_doublets.obs['leiden_res0_25'] == '1'), :]
print(neurons_new_1)
neurons_new_5 = adata_donor_assigned_no_doublets[(adata_donor_assigned_no_doublets.obs['leiden_res0_25'] == '5'), :]
print(neurons_new_5)

In [ ]:
ratio_new_old_neurons = (((len(neurons_new_1.obs) + len(neurons_new_5.obs)) / len(neurons_old.obs)) * 100) - 100
print("We have found", ratio_new_old_neurons, "percent more neurons than in the original count.")

In [ ]:
import anndata as ad
neurons_new = ad.concat([neurons_new_1, neurons_new_5], axis=0)

Comparing the old neuron clusters with the new cells in these clusters.

In [ ]:
sc.pl.umap(
    neurons_new, save=".png",
    color=["bucket"]
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plt.show(sc.pl.umap(adata_donor_assigned_no_doublets, color=["cluster_label"], ax=ax))

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, ["SNAP25", "GABRB2", "SYT1", "CUX2", "VIP", "RORB", "THY1", "TLE4", "NRGN"], groupby='leiden_res0_25')

Plotting a small overview of all leiden 0.25 clusters with 3 dotplots with different genes included.

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, ["SNAP25",
                                                 "GABRB2",
                                                 "SYT1",
                                                 "CUX2",
                                                 "VIP",
                                                 "RORB",
                                                 "THY1",
                                                 "TLE4",
                                                 "NRGN",
                                                 "SLC17A7",
                                                 "GAD2",
                                                 "AQP4",
                                                 "GFAP",
                                                 "GLUL",
                                                 "PTPRC",
                                                 "PDGFRA",
                                                 "OPALIN",
                                                 "PLP1",
                                                 "FTL",
                                                 "PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
                                                ], color_map='Greens',groupby='leiden_res0_25')

In [ ]:
sc.pl.umap(
    adata,
    color=["leiden_res0_25", "bucket"],
    legend_loc="on data", save="leiden025_bucket.png"
)

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, [
"AQP4",
"GFAP",
"GLUL",
"IGHG1",
"ICAM2",
"SLC17A7",
"GAD2",
"PTPRC",
"MSR1",
"FTL",
"SNAP25",
"GABRB2",
"SYT1",
"CUX2",
"VIP",
"RORB",
"THY1",
"TLE4",
"VIP",
"NRGN",
"PDGFRA",
"OPALIN",
"PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
"PLP1",
"MBP"
                                                ], color_map='Greens', groupby='leiden_res0_25')

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, [
"AQP4",
"GFAP",
"GLUL",
"IGHG1",
"ICAM2",
"SLC17A7",
"GAD2",
"PTPRC",
"MSR1",
"FTL",
"SNAP25",
"GABRB2",
"SYT1",
"CUX2",
"VIP",
"RORB",
"THY1",
"TLE4",
"VIP",
"NRGN",
"PDGFRA",
"OPALIN",
"PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
"PLP1",
"MBP"
                                                ], color_map='Greens', groupby='leiden_res0_25')

In [ ]:
adata_donor_assigned_no_doublets_nan_rows = adata_donor_assigned_no_doublets.obs['bucket'].isna()
adata_donor_assigned_no_doublets_nan = adata_donor_assigned_no_doublets[adata_donor_assigned_no_doublets_nan_rows, :]
adata_donor_assigned_no_doublets_nan

In [ ]:
adata_donor_assigned_no_doublets_not_nan_rows = adata_donor_assigned_no_doublets.obs['bucket'].notna()
adata_donor_assigned_no_doublets_not_nan = adata_donor_assigned_no_doublets[adata_donor_assigned_no_doublets_not_nan_rows, :]
adata_donor_assigned_no_doublets_not_nan

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets_not_nan, [
"AQP4",
"GFAP",
"GLUL",
"IGHG1",
"ICAM2",
"SLC17A7",
"GAD2",
"PTPRC",
"MSR1",
"FTL",
"SNAP25",
"GABRB2",
"SYT1",
"CUX2",
"VIP",
"RORB",
"THY1",
"TLE4",
"VIP",
"NRGN",
"PDGFRA",
"OPALIN",
"PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
"PLP1",
"MBP"
                                                ], color_map='Reds', groupby='cluster_label')

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets_nan, [
"AQP4",
"GFAP",
"GLUL",
"IGHG1",
"ICAM2",
"SLC17A7",
"GAD2",
"PTPRC",
"MSR1",
"FTL",
"SNAP25",
"GABRB2",
"SYT1",
"CUX2",
"VIP",
"RORB",
"THY1",
"TLE4",
"VIP",
"NRGN",
"PDGFRA",
"OPALIN",
"PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
"PLP1",
"MBP"
                                                ], color_map='Blues', groupby='leiden')

Big overview for cell Markers.

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, ['GJA1', 'AQP4', 'SLC1A2', 'GPC5', 'GFAP',
'CD44', 'GLUL', 'KCNJ10', 'BCL6', 'HSP90AA1',
'FAIM2', 'ATF4', 'UBB', 'BCAS1', 'SGMS1',
'KCNJ10', 'SEMA6A', 'GLDN', 'FOS', 'LINC01088',
'SLC1A2', 'SLC1A3', 'RFX4', 'GPC5', 'CD44',
'IGHG1', 'CLDN5', 'ICAM2', 'SLC17A7', 'GAD2',
'PTPRC', 'MSR1', 'P2RY12', 'MBP', 'ASAH1',
'ACSL1', 'DPYD', 'CD68', 'FTL', 'SPP1',
'SYNDIG1', 'KCNQ3', 'AIF1', 'SNAP25', 'GABRB2',
'SYT1', 'CUX2', 'VIP', 'RORB', 'THY1',
'TLE4', 'VIP', 'PPIA', 'RBFOX3', 'SATB2',
'GAD1', 'PVALB', 'SST', 'SV2C', 'NRGN',
'PDGFRA', 'BCAN', 'SOX6', 'CDH20', 'RBFOX1',
'LURAP1L-AS1', 'CDH19', 'KLK6', 'GJB1', 'OPALIN',
'LINC00844', 'PLP1',
'FTL', 'FTH1', 'B2M', 'HLA-C', 'CNP',
'PDGFRB', 'NOTCH3', 'MZB1', 'SDC1', 'PRDM1',
'LAMA2', 'SKAP1', 'PRF1', 'ITGA4', 'TRAC',
'CXCR4', 'CCL5', 'PLP1', 'MBP', 'CLDN11',
'ST18', 'MSR1', 'TFEC', 'RUNX2', 'SRGN',
'CD14', 'FTL', 'C1QA', 'C1QB', 'C1QC',
'P2RY12', 'PTPRJ', 'RUNX1', 'BACH1',
'TGFBR2', 'IFI16', 'IL18', 'CSF1R', 'C3',
'SYK', 'SRGAP2', 'FGD4', 'ALOX5',
'CTSC', 'SPP1', 'CD74', 'HLA-DRB1',
'HLA-DRA', 'MAFG', 'NFE2L2',
'MAT2A', 'CSF2RB', 'CSF2RA', 'CTLA4',
'TIGIT', 'CD69', 'IL23R', 'KLRB1',
'CD19', 'MS4A1', 'HBG2', 'HBB', 'RPS6',
'RPLP1', 'TFRC', 'HBA1', 'HBA2', 'IGHM',
'MKI67', 'TUBA1B', 'STMN1', 'IGHA1', 'IGHG1',
'IGHG2', 'IGHG3', 'IGHG4', 'TNFRSF13B', 'XBP1', 'SLAMF7']

                                                , color_map='Greens', groupby='leiden')

Replotted leiden and bucket for reference.

In [ ]:
sc.pl.umap(
    adata_donor_assigned_no_doublets,
    color=["leiden", "bucket"],
    legend_loc="on data",
)

In [ ]:
adata.obs['newold'] = np.where(pd.isna(adata.obs['bucket']), 'new', 'old')
adata.obs["leiden_new_old"]=adata.obs['leiden'].str.cat(adata.obs['newold'], sep='_')
adata.obs["leiden_res0_25_new_old"]=adata.obs['leiden_res0_25'].str.cat(adata.obs['newold'], sep='_')
adata_donor_assigned_no_doublets.obs['newold'] = np.where(pd.isna(adata_donor_assigned_no_doublets.obs['bucket']), 'new', 'old')
adata_donor_assigned_no_doublets.obs["leiden_new_old"]=adata_donor_assigned_no_doublets.obs['leiden'].str.cat(adata_donor_assigned_no_doublets.obs['newold'], sep='_')
adata_donor_assigned_no_doublets.obs["leiden_res0_25_new_old"]=adata_donor_assigned_no_doublets.obs['leiden_res0_25'].str.cat(adata_donor_assigned_no_doublets.obs['newold'], sep='_')

In [ ]:
adata.obs

Showing dotplots comparing new and old cells.

In [ ]:
marker_genes = ["AQP4",
"GFAP",
"GLUL",
"IGHG1",
"ICAM2",
"SLC17A7",
"GAD2",
"PTPRC",
"MSR1",
"FTL",
"SNAP25",
"GABRB2",
"SYT1",
"CUX2",
"VIP",
"RORB",
"THY1",
"TLE4",
"VIP",
"NRGN",
"PDGFRA",
"OPALIN",
"PDGFRB",
"NOTCH3",
"MZB1",
"SDC1",
"PRDM1",
"LAMA2",
"PLP1",
"MBP"]
sc.pl.dotplot(adata_donor_assigned_no_doublets, marker_genes, color_map='Greens', save="new_old_leiden025.png", groupby='leiden_res0_25_new_old')

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, marker_genes, color_map='Greens', save="new_old.png", groupby='leiden_new_old')

In [ ]:
sc.pl.dotplot(adata_donor_assigned_no_doublets, ['GJA1', 'AQP4', 'SLC1A2', 'GPC5', 'GFAP',
'CD44', 'GLUL', 'KCNJ10', 'BCL6', 'HSP90AA1',
'FAIM2', 'ATF4', 'UBB', 'BCAS1', 'SGMS1',
'KCNJ10', 'SEMA6A', 'GLDN', 'FOS', 'LINC01088',
'SLC1A2', 'SLC1A3', 'RFX4', 'GPC5', 'CD44',
'IGHG1', 'CLDN5', 'ICAM2', 'SLC17A7', 'GAD2',
'PTPRC', 'MSR1', 'P2RY12', 'MBP', 'ASAH1',
'ACSL1', 'DPYD', 'CD68', 'FTL', 'SPP1',
'SYNDIG1', 'KCNQ3', 'AIF1', 'SNAP25', 'GABRB2',
'SYT1', 'CUX2', 'VIP', 'RORB', 'THY1',
'TLE4', 'VIP', 'PPIA', 'RBFOX3', 'SATB2',
'GAD1', 'PVALB', 'SST', 'SV2C', 'NRGN',
'PDGFRA', 'BCAN', 'SOX6', 'CDH20', 'RBFOX1',
'LURAP1L-AS1', 'CDH19', 'KLK6', 'GJB1', 'OPALIN',
'LINC00844', 'PLP1',
'FTL', 'FTH1', 'B2M', 'HLA-C', 'CNP',
'PDGFRB', 'NOTCH3', 'MZB1', 'SDC1', 'PRDM1',
'LAMA2', 'SKAP1', 'PRF1', 'ITGA4', 'TRAC',
'CXCR4', 'CCL5', 'PLP1', 'MBP', 'CLDN11',
'ST18', 'MSR1', 'TFEC', 'RUNX2', 'SRGN',
'CD14', 'FTL', 'C1QA', 'C1QB', 'C1QC',
'P2RY12', 'PTPRJ', 'RUNX1', 'BACH1',
'TGFBR2', 'IFI16', 'IL18', 'CSF1R', 'C3',
'SYK', 'SRGAP2', 'FGD4', 'ALOX5',
'CTSC', 'SPP1', 'CD74', 'HLA-DRB1',
'HLA-DRA', 'MAFG', 'NFE2L2',
'MAT2A', 'CSF2RB', 'CSF2RA', 'CTLA4',
'TIGIT', 'CD69', 'IL23R', 'KLRB1',
'CD19', 'MS4A1', 'HBG2', 'HBB', 'RPS6',
'RPLP1', 'TFRC', 'HBA1', 'HBA2', 'IGHM',
'MKI67', 'TUBA1B', 'STMN1', 'IGHA1', 'IGHG1',
'IGHG2', 'IGHG3', 'IGHG4', 'TNFRSF13B', 'XBP1', 'SLAMF7']

                                                , color_map='Greens', groupby='leiden_res0_25')

In [ ]:
marker_genes_dict = {
    'Neurons': ["SNAP25", "GABRB2", "SYT1", "CUX2", "VIP", "RORB", "THY1", "TLE4", "VIP", "NRGN"],
    'Excitatory Neurons': ['SLC17A7'],
    'Inhibitory Neurons': ['GAD2'],
    'Astrocytes': ["AQP4", "GFAP", "GLUL"],
    'Oligodendrocytes': ["PDGFRA", "OPALIN", "PLP1", "MBP"],
    'Endothelial cells': "ICAM2",
    'Leukocytes': "PTPRC"
}
sc.pl.dotplot(adata_donor_assigned_no_doublets, marker_genes_dict,groupby='leiden_res0_25_new_old')

Looking at the different doublet scores and n_genes to figure out differences as seen in the violin plot above.

In [ ]:
sc.pl.scatter(adata, x='n_genes', y='doublet_score')

In [ ]:
sc.pl.scatter(adata, x='n_genes', y='paper_doublet_scores')

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
print(adata.obs.keys())

In [ ]:
sc.pl.scatter(adata, x='doublet_score', y='paper_doublet_scores', color='doublet_classification_pipeline')

In [ ]:
sc.pl.scatter(adata_old, x='doublet_score', y='paper_doublet_scores', color='doublet_classification_pipeline')

In [ ]:
sc.pl.scatter(adata, x='doublet_score', y='paper_doublet_scores', color='n_genes')

In [ ]:
adata.obs['paper_predicted_doublets'] = adata.obs['paper_predicted_doublets'].astype(str)
adata.write_h5ad('anndata_post_QC_ms_data_analysis_lennard_24042023_streamlined_with_hashing_info.h5ad')

In [ ]:
sc.tl.rank_genes_groups(adata, 'bucket', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
print(decontamination_matrix.T)